<a href="https://colab.research.google.com/github/HSS107048212/Graph-attention-networks-applied-in-patent/blob/main/Neil_GATcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
import os

In [ ]:
patent_data = pd.read_excel('c9-2.xlsx', header=1) #這一個要放蒐集下來的核心專利
patent_data_family = pd.read_excel('c9-family.xlsx', header=1) #這一個要放核心專利的專利家族和集
patent_data = patent_data.fillna('')

#下面是將專利家族的引用關係複製到核心專利那個上面，由於在建構引用網路時重複的關係並不會被計算到，因此採直接複製
#下面這段是在複製DWPI family的資料

for i in patent_data.index: #整理 Cited Refs - Patent 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = patent_data_family['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index: # 整理 Citing Patents 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = patent_data_family['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c


#下面是做一樣的事情，只是變成複製INPADC的資料，單純因為懶，所以分成兩個比較方便
INPADC1 = pd.read_excel('INPADC1.xlsx', header=1)
INPADC2 = pd.read_excel('INPADC2.xlsx', header=1)
INPADC3 = pd.read_excel('INPADC3.xlsx', header=1)
INPADC4 = pd.read_excel('INPADC4.xlsx', header=1)
INPADC = pd.concat([INPADC1,INPADC2])
INPADC = pd.concat([INPADC,INPADC3])
INPADC = pd.concat([INPADC,INPADC4])
INPADC.index = list(range(len(INPADC)))

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = INPADC['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'INPADOC Family Members']):
        temp = patent_data.loc[i,'INPADOC Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = INPADC['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c

#最後將引用關係合併之後的資料集輸出為excel，在後續分析時要用
patent_data.to_excel('c9-append.xlsx')

<ipython-input-4-b012f9d53a5c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Cited Refs - Patent'][i] = c
<ipython-input-4-b012f9d53a5c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Citing Patents'][i] = c


In [ ]:
patent_data

,Publication Number,Title (English),Assignee/Applicant,Publication Year,Publication Date,Abstract (English),Abstract - DWPI,Cited Refs - Patent,Citing Patents,Count of Citing Patents,Count of Cited Refs - Patent,IPC Current Full (4 Characters),INPADOC Family Members,DWPI Family Members
0,TW201818743A,ROUTING METHOD AND WIRELESS NODE FOR WIRELESS ...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2018,2018-05-16,A routing method for a wireless mesh network a...,The method involves performing (S610) route al...,US7159035B2 | US8488589B2 | US8948046B2 | US90...,CN113259940A | CN113259940B | TWI727786B|CN114...,3,4,"H04L, H04W",TW201818743A | TWI604744B | US10250503B2 | US...,US20180123957A1 | TWI604744B | TW201818743A |...
1,TW201946421A,TRANSPARENT BRIDGING OVER MESH NETWORK,"GUIZHOU DINGTONGXIN TECHNOLOGIES LTD.,CN | 大陸商...",2019,2019-12-01,A method of transparent bridging over mesh net...,The method involves receiving a packet from an...,US20170034760A1 | US8730809B2 | US20130100872A...,,0,4,H04L,TW201946421A | CN110417659A | TWI731237B,CN110417659A | TW201946421A | TWI731237B
2,TW201715902A,MANAGEMENT METHOD FOR WIRELESS NETWORK AND NET...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2017,2017-05-01,A wireless network management method for a wir...,A wireless network management method for a wir...,US20110255570A1 | US20140153551A1 | US20140119...,WO2019127204A1|WO2019127204A1|US10356759B2 | U...,1,3,H04W,TW201715902A | TWI568279B | US20170118752A1 |...,TW201715902A | TWI568279B | US9999045B2 | US2...
3,TW202245502A,METHOD AND SYSTEM FOR UPDATING DEVICE FIRMWARE...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A firmware update method. On a wireless mesh n...,A firmware update method. On a wireless mesh n...,CN106713047A | CN102158851B | CN103685530A|CN1...,,0,3,"H04W, H04L, G06F",,TW202245502A | TWI784506B
4,TW202245501A,METHOD AND SYSTEM FOR REMOTELY CONTROLLING FIR...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A method or system for remotely controlling th...,A method or system for remotely controlling th...,CN103685530A | CN106713047A | CN102158851B|CN1...,,0,3,"H04W, H04L, G06F",,TW202245501A | TWI784505B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,TW202114382A,ROUTER AND ROUTING METHOD,"REALTEK SEMICONDUCTOR CORPORATION,TW | 瑞昱半導體股份...",2021,2021-04-01,The application discloses a router coupled to ...,The router (100) has a switch module (112) pro...,US8565240B2 | US20030016628A1 | WO2013081511A1...,,0,4,H04L,TW202114382A | TWI730476B | US11206568B2 | US2...,US20210092633A1 | TW202114382A | TWI730476B | ...
92,TW202113749A,SYSTEM FOR INTEGRATED MANAGEMENT OF VIRTUAL AN...,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-04-01,The present invention is a system for integrat...,The present invention is a system for integrat...,TW201527991A | TW201911045A | TW200611112A | T...,,0,5,"G06Q, H04L",TW202113749A | TWI709941B,TWI709941B | TW202113749A
93,TW202121869A,MANAGEMENT SYSTEM AND MANAGEMENT METHOD OF VNF,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-06-01,A management system and management method of V...,A management system and management method of V...,TW201909601A | WO2015106822A1 | US20190058636A...,|TWI789014B | TWI795123B,0,7,H04L,TW202121869A | TWI707561B,TWI707561B | TW202121869A
94,TW202121917A,CHANNEL LOADING PRE-ADJUSTING SYSTEM FOR 5G WI...,"ALPHA NETWORKS INC.,TW | 明泰科技股份有限公司,TW",2021,2021-06-01,A channel loading pre-adjusting system for 5G ...,The channel loading pre-adjusting system (10) ...,TW201212693A | US20170332270A1 | US20080134194...,|CN113965946A,0,3,H04W,TW202121917A | CN112839357A | CN112839357B | T...,US20210160734A1 | CN112839357A | TWI701956B | ...


In [ ]:
import pandas as pd
import numpy as np
import copy

import os

#從Derwent下載下來的資料，核心專利、核心專利引用的專利跟引用核心專利的專利
#patent_data = pd.read_excel('1215_append.xlsx', header=0) #這個是要使用合併引用關係之後的核心資料集
#cited_data = pd.read_excel('1215_cited.xlsx', header=1)
#citing_data = pd.read_excel('1215_citing.xlsx', header=1)

# Neil學長的 已经将Excel 文件转换为了 CSV 格式，并且文件名保持一致，只是扩展名变为了 .csv
patent_data = pd.read_csv('1215_append.csv', header=0, encoding='utf-8') # 这个是要使用合并引用关系之后的核心数据集
cited_data = pd.read_csv('1215_cited.csv', header=2, encoding='utf-8')
citing_data = pd.read_csv('1215_citing.csv', header=2, encoding='utf-8')

#cited_data = cited_data.iloc[3:]
#citing_data = citing_data.iloc[3:]
print(patent_data.columns)
print(cited_data.columns)
print(citing_data.columns)


#整理資料，保留特定欄位並合併三個資料集為全部使用到的大資料集。
temp_patent = patent_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents'] ]
temp_cited = cited_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
temp_citing = citing_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
all_patent = pd.concat([temp_patent,temp_cited])
all_patent = pd.concat([all_patent,temp_citing])
all_patent = all_patent.dropna(subset=['Title (English)','Abstract (English)','Abstract - DWPI']) #將三行都是空值的列刪除
#刪除重複值並用Publication Year排序
all_patent = all_patent.drop_duplicates()

# 将 'Publication Year' 列转换为整数，无法转换的设置为 NaN
all_patent['Publication Year'] = pd.to_numeric(all_patent['Publication Year'], errors='coerce')

# 现在可以排序了，因为所有数据都是整数类型
all_patent = all_patent.sort_values('Publication Year')

# 如果你想要删除 'Publication Year' 列中的 NaN 值
all_patent = all_patent.dropna(subset=['Publication Year'])

all_patent = all_patent.sort_values('Publication Year')
all_patent1 = all_patent.drop_duplicates(subset=['Publication Number'])
all_patent1.index = list(range(len(all_patent1)))
#合併會使用到的專利文字資料，並在原資料及創造一個新的欄位名叫text存放
text = []
for i in all_patent1.index:

  str1 = str(all_patent1.loc[i,'Title (English)'])+' '+str(all_patent1.loc[i,'Abstract (English)'])+' '+str(all_patent1.loc[i,'Abstract - DWPI'])

  str2 = ''.join(str1.split('nan'))
  text.append(str2)
text = np.array(text)
all_patent1['text'] = text

#有text的專利資料集先存起來備用
all_patent1.to_excel('IoV new.xlsx')

Index(['Unnamed: 0', 'Publication Number', 'Title (English)',
       'Assignee/Applicant', 'Publication Year', 'Publication Date',
       'Abstract (English)', 'Abstract - DWPI', 'Cited Refs - Patent',
       'Citing Patents', 'Count of Citing Patents',
       'Count of Cited Refs - Patent', 'IPC Current Full (4 Characters)',
       'INPADOC Family Members', 'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Patents',
       'Count of Citing Patents', 'Count of Cited Refs - Patent',
       'IPC Current Full (4 Characters)', 'INPADOC Family Members',
       'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Pate